**Homework 9:**



# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "HW9a"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
from rdkit.DataStructs import ConvertToNumpyArray

from rdkit.Chem import PandasTools
import openchem

# Load the data

In [3]:
#
datapath = os.path.join(".", "data/raw/esol.csv")
esol_data = pd.read_csv(datapath)
esol_data.head()

,cano_smiles,activity,group
0,Cc1occc1C(=O)Nc1ccccc1,-3.30,train
1,CC(C)=CCCC(C)=CC=O,-2.06,train
2,c1ccc2c(c1)ccc1c2ccc2c3ccccc3ccc21,-7.87,train
3,c1ccsc1,-1.33,train
4,c1ccc2scnc2c1,-1.50,train


## Data preprocessing

In [4]:
esol_data.describe()

,activity
count,1127.000000
mean,-3.052010
std,2.096392
min,-11.600000
25%,-4.321000
50%,-2.860000
75%,-1.600000
max,1.580000


In [5]:
#Generate data exploration
esol_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127 entries, 0 to 1126
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cano_smiles  1127 non-null   object 
 1   activity     1127 non-null   float64
 2   group        1127 non-null   object 
dtypes: float64(1), object(2)
memory usage: 26.5+ KB


In [6]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

model = AutoModelForMaskedLM.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

/ext3/miniconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
tokenizer(esol_data["cano_smiles"][0:3].tolist(), return_tensors="pt", padding=True)

{'input_ids': tensor([[  0, 267,  21, 423,  21,  39, 263,  51,  13, 277,  21, 269,  21,   2,
           1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  0, 262,  12,  39, 295, 285,  12,  39, 295, 262,  33,  51,   2,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  0,  71,  21, 264,  22,  71,  12,  71,  21,  13, 264,  21,  71,  22,
         264,  22,  71,  23, 269,  23, 264, 309,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [8]:
tokenizer.cls_token

'<s>'

In [9]:
tokenizer.cls_token_id

0

In [10]:
esol_data["embedding"] = esol_data["cano_smiles"].map(lambda smiles: model(**tokenizer(smiles, return_tensors="pt",
                                                                                       padding=True))["logits"][0, 0, :])

In [11]:
data_train = esol_data[esol_data["group"]=="train"]
data_valid = esol_data[esol_data["group"]=="valid"]
data_test = esol_data[esol_data["group"]=="test"]

In [12]:
import torch

X_train = torch.stack(data_train["embedding"].tolist()).detach().numpy()
y_train = data_train["activity"].values
X_valid = torch.stack(data_valid["embedding"].tolist()).detach().numpy()
y_valid = data_valid["activity"].values
X_test = torch.stack(data_test["embedding"].tolist()).detach().numpy()
y_test = data_test["activity"].values

In [13]:
X_train.shape

(901, 767)

### XGBoost

In [14]:
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)
model = xgb.fit(X_train, y_train)

In [15]:
from sklearn.metrics import mean_squared_error

y_train_pred = model.predict(X_train)
xgb_mse = mean_squared_error(y_train, y_train_pred)
xgb_rmse = np.sqrt(xgb_mse)
print("train RMSE ", xgb_rmse)

train RMSE  0.02894841276140909


In [16]:
y_valid_pred = model.predict(X_valid)
xgb_mse = mean_squared_error(y_valid, y_valid_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_rmse

1.4842408532879732

In [17]:
y_test_pred = model.predict(X_test)
xgb_mse = mean_squared_error(y_test, y_test_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_rmse

1.3184776770404631

In [19]:
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'gamma': [0.001, 0.01, 0.1, 0.7],
    'min_child_weight': range(1, 10, 4),
    'subsample': np.arange(0.1, 1.0, 0.3),
    'colsample_bytree': np.arange(0.1, 1.0, 0.4),
    'max_depth': range(3, 10, 4),
    'n_estimators': range(200, 1000, 400)
        }

from sklearn.model_selection import ParameterGrid
grid = ParameterGrid(param_grid)

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
from sklearn.metrics import mean_squared_error

best_model = None
best_rmse=np.inf
best_param = {}
for this_param in tqdm(grid):
    this_model = XGBRegressor(**this_param, random_state=42)
    this_model.fit(X_train, y_train)
#     print(this_model.predict(X_valid))
    y_pred = this_model.predict(X_valid)
    xgb_mse = mean_squared_error(y_valid, y_pred)
    xgb_rmse = np.sqrt(xgb_mse)
    if best_rmse > xgb_rmse:
        best_model = this_model
        best_rmse = xgb_rmse
        best_param = this_param

print(best_param)
print(best_rmse)

100%|██████████| 1296/1296 [3:28:00<00:00,  9.63s/it]  

{'colsample_bytree': 0.1, 'gamma': 0.7, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 9, 'n_estimators': 200, 'subsample': 0.7000000000000001}
1.2566624365831074


### Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
lin = RandomForestRegressor()
lin.fit(X_train, y_train)
model = lin

In [21]:
from sklearn.metrics import mean_squared_error

y_train_pred = model.predict(X_train)
xgb_mse = mean_squared_error(y_train, y_train_pred)
xgb_rmse = np.sqrt(xgb_mse)
print("train RMSE ", xgb_rmse)

y_valid_pred = model.predict(X_valid)
xgb_mse = mean_squared_error(y_valid, y_valid_pred)
xgb_rmse = np.sqrt(xgb_mse)
print("valid RMSE ", xgb_rmse)

y_test_pred = model.predict(X_test)
xgb_mse = mean_squared_error(y_test, y_test_pred)
xgb_rmse = np.sqrt(xgb_mse)
print("test RMSE ", xgb_rmse)

train RMSE  0.48464012001713086
valid RMSE  1.4336586177157944
test RMSE  1.21640373547409


In [22]:
param_grid = {
        'n_estimators': [10, 50, 200, 500, 1000],  
        'max_depth': range(3, 12),
        'min_samples_leaf': [1, 3, 10, 50],
        'min_impurity_decrease': [0, 0.01] ,
        'max_features':  ['sqrt', 'log2', 0.8]
        }

from sklearn.model_selection import ParameterGrid
grid = ParameterGrid(param_grid)

len(grid)
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

best_model = None
best_rmse=np.inf
best_param = {}
for this_param in tqdm(grid):
    this_model = RandomForestRegressor(**this_param, random_state=42)
    this_model.fit(X_train, y_train)
    forest_mse = mean_squared_error(y_valid, this_model.predict(X_valid))
    forest_rmse = np.sqrt(forest_mse)
    if best_rmse > forest_rmse:
        best_model = this_model
        best_rmse = forest_rmse
        best_param = this_param
        
print(best_param)
print(best_rmse)

100%|██████████| 1080/1080 [3:53:19<00:00, 12.96s/it]  

{'max_depth': 10, 'max_features': 0.8, 'min_impurity_decrease': 0, 'min_samples_leaf': 1, 'n_estimators': 200}
1.4215655424302922


### Dense NN

In [23]:
import torch.nn as nn
nn_dense = nn.Sequential(
    nn.ReLU(),
    nn.Linear(767, 384),
    nn.ReLU(),
    nn.Linear(384, 190),
    nn.ReLU(),
    nn.Linear(190, 1)
)

In [24]:
import torch
from tqdm import tqdm
reg_model = nn_dense
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(reg_model.parameters(), lr=1e-3)
n_epoches = 10
# data_train.shape[0]
index_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.arange(100))
                                           , batch_size=8, shuffle=True)
for i in range(n_epoches):
    for train_idx in (index_loader):
        optimizer.zero_grad()
        train_idx = train_idx[0].tolist()
        pred = reg_model(torch.as_tensor(X_train[train_idx]))
        label = torch.as_tensor(data_train["activity"][train_idx].values).float()
        loss = loss_fn(pred.reshape(-1), label.reshape(-1))
        loss.backward()
        optimizer.step()
        
    val_loss = 999
    with torch.no_grad():
        pred = reg_model(torch.as_tensor(X_valid))
        label = torch.as_tensor(data_valid["activity"].values)
        val_loss = loss_fn(pred.reshape(-1), label.reshape(-1))
    print("train loss:", loss, "valid loss:", val_loss)

train loss: tensor(0.6487, grad_fn=<MseLossBackward>) valid loss: tensor(3.5886, dtype=torch.float64)
train loss: tensor(3.3479, grad_fn=<MseLossBackward>) valid loss: tensor(3.6337, dtype=torch.float64)
train loss: tensor(1.9942, grad_fn=<MseLossBackward>) valid loss: tensor(3.6187, dtype=torch.float64)
train loss: tensor(1.5300, grad_fn=<MseLossBackward>) valid loss: tensor(3.2085, dtype=torch.float64)
train loss: tensor(0.4254, grad_fn=<MseLossBackward>) valid loss: tensor(4.0099, dtype=torch.float64)
train loss: tensor(1.5355, grad_fn=<MseLossBackward>) valid loss: tensor(3.0528, dtype=torch.float64)
train loss: tensor(0.3423, grad_fn=<MseLossBackward>) valid loss: tensor(3.1474, dtype=torch.float64)
train loss: tensor(0.9480, grad_fn=<MseLossBackward>) valid loss: tensor(3.1459, dtype=torch.float64)
train loss: tensor(0.4106, grad_fn=<MseLossBackward>) valid loss: tensor(3.1181, dtype=torch.float64)
train loss: tensor(0.7483, grad_fn=<MseLossBackward>) valid loss: tensor(3.0599, d